## SET CUDA

In [3]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Get CSV DATA

In [4]:
import csv
from unidecode import unidecode
latitude = []
longitude = []
city_name = []
county_name = []
city_population = []

with open('Datasets/ro.csv', newline='') as csvfile:
    DictReader = csv.DictReader(csvfile)
    ### Get latitude and longitude from csv
    for row in DictReader:
        city = unidecode(row['city'])
        city = city.replace("AY", 's')
        city = city.replace("APS", 't')
        city = city.replace("Af", 'a')
        city = city.replace("AC/", 'a')
        city_name = city_name + [city]
        county = unidecode(row['admin_name'])
        county = county.replace("AY", 's')
        county = county.replace("APS", 't')
        county = county.replace("Af", 'a')
        county = county.replace("AC/", 'a')
        county_name = county_name + [county]
        latitude = latitude + [row['lat']]
        longitude = longitude + [row['lng']]
        city_population = city_population + [row['population']]

# GET AVERAGE SHORTEST PATH (meters) between hospitals and streets

In [5]:
import networkx as nx
import osmnx as ox
import numpy as np
import random
from functools import lru_cache

# Create empty lists to store data
average_shortest_path = []

for i in range(1, len(city_name)):
    try:
        if i != 0:
            location = city_name[i]+","+county_name[i]
        else:
            location = city_name[i]
        # Get the street network graph
        G = ox.graph_from_place(location, network_type="drive")

        # Get the hospitals data
        hospitals = ox.features.features_from_place(location, tags={"amenity": "hospital", "amenity":"clinic", "healthcare":"hospital", "emergency":"yes"})

        # Identify hospital nodes
        hospital_nodes = []
        for _, hospital in hospitals.iterrows():
            centroid = hospital.geometry.centroid
            latitude, longitude = centroid.y, centroid.x
            nearest_node = ox.distance.nearest_nodes(G, longitude, latitude)
            hospital_nodes.append(nearest_node)

        # Get a list of all street nodes (excluding hospital nodes)
        street_nodes = [node for node in G.nodes() if node not in hospital_nodes]

        # Randomly sample 200 street nodes
        sampled_street_nodes = random.sample(street_nodes, 200)

        # Caching mechanism for ox.shortest_path results
        @lru_cache(maxsize=None)
        def get_shortest_path_length(source, target):
            path = ox.shortest_path(G, source, target, weight='length')
            if path is None or len(path) <= 1:
                return float('inf')
            return sum(ox.utils_graph.get_route_edge_attributes(G, path, 'length'))

        # Calculate the shortest path lengths from each sampled street node to the closest hospital node
        shortest_path_lengths = []
        for street_node in sampled_street_nodes:
            min_path_length = float('inf')
            for hospital_node in hospital_nodes:
                try:
                    path_length = get_shortest_path_length(hospital_node, street_node)
                    if path_length < min_path_length:
                        min_path_length = path_length
                except (nx.NetworkXNoPath, KeyError):
                    pass  # If there is no path or other errors, we skip this pair
            if min_path_length < float('inf'):
                shortest_path_lengths.append(min_path_length)

        # Calculate the average shortest path length - METERS
        if shortest_path_lengths:
            average_shortest_path_length = np.mean(shortest_path_lengths)
            #print(f"{location} - average shortest path length from 200 random street nodes to the closest hospital: {average_shortest_path_length} meters")
            average_shortest_path.append(average_shortest_path_length)
        else:
            print("No valid shortest paths found.")
    except Exception as e:
        print(f"Error processing {location}: {e}")

C:\Users\Adrian\AppData\Local\Temp\ipykernel_20560\26703182.py:42: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `routing.route_to_gdf` function instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  return sum(ox.utils_graph.get_route_edge_attributes(G, path, 'length'))


KeyboardInterrupt: 

## GET AVERAGE TIME BETWEEN HOSPITAL AND STREETS

In [ ]:
import random
import networkx as nx
import osmnx as ox
import numpy as np



# Define typical speeds (in km/h) for different street types
street_speeds = {
    'motorway': 90,
    'trunk': 70,
    'primary': 50,
    'secondary': 45,
    'tertiary': 35,
    'residential': 25,
    'unclassified': 20,
    'service': 10
}

# Create empty lists to store data
average_shortest_time = []

for i in range(1, len(city_name)):
    try:
        if i != 0:
            location = city_name[i]+","+county_name[i]
        else:
            location = city_name[i]
            
        # Define the location and graph
            
        G = ox.graph_from_place(location, network_type="drive")

        # Get the hospitals data
        hospitals = ox.features.features_from_place(location, tags={"amenity": "hospital", "amenity":"clinic", "healthcare":"hospital", "emergency":"yes"})

        # Extract hospital nodes
        hospital_nodes = []
        for _, hospital in hospitals.iterrows():
            point = hospital.geometry.centroid
            nearest_node = ox.distance.nearest_nodes(G, point.x, point.y)
            hospital_nodes.append(nearest_node)
        hospital_nodes = list(set(hospital_nodes))

        # Ensure travel_time is calculated for each edge
        default_speed_kph = 25
        for u, v, k, data in G.edges(data=True, keys=True):
            highway = data.get('highway')
            if isinstance(highway, list):
                highway = highway[0]  # If highway type is a list, take the first element
            speed_kph = street_speeds.get(highway, default_speed_kph)
            if 'length' in data:
                length_km = data['length'] / 1000  # Convert length to kilometers
                travel_time_hours = length_km / speed_kph  # Travel time in hours
                travel_time_seconds = travel_time_hours * 3600  # Convert to seconds
                # Introduce random delay to simulate real-world conditions
                random_delay = np.random.uniform(15, 30)  # Random delay between 5 and 15 seconds
                data['travel_time'] = travel_time_seconds + random_delay

        # Extract street nodes
        street_nodes = list(G.nodes)
        sampled_street_nodes = random.sample(street_nodes, 200)

        # Compute shortest paths for each street node to the closest hospital node
        shortest_travel_times = []
        for street_node in sampled_street_nodes:
            min_travel_time = float('inf')
            for hospital_node in hospital_nodes:
                try:
                    travel_time = sum(ox.utils_graph.get_route_edge_attributes(G, ox.shortest_path(G, street_node, hospital_node, weight='travel_time'), 'travel_time'))
                    if travel_time < min_travel_time:
                        min_travel_time = travel_time
                except Exception as e:
                    continue
            if min_travel_time < float('inf'):
                shortest_travel_times.append(min_travel_time)

        # Calculate the average travel time
        if shortest_travel_times:
            average_travel_time = sum(shortest_travel_times) / len(shortest_travel_times)
            travel_time_minutes = (int)(average_travel_time/60)
            travel_time_seconds = average_travel_time%60
            #print(f"{location} - Number of hospitals/clinics: {len(hospital)}")
            #print(f"{location} - Average shortest travel time from 200 streets to the closest hospital: {travel_time_minutes} minutes {travel_time_seconds:.1f} seconds")
            average_shortest_time.append(average_travel_time)
        else:
            print("No valid shortest travel times found.")
    except Exception as e:
        print(f"Error processing {location}: {e}")


C:\Users\Adrian\AppData\Local\Temp\ipykernel_20956\1004238746.py:70: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `routing.route_to_gdf` function instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  travel_time = sum(ox.utils_graph.get_route_edge_attributes(G, ox.shortest_path(G, street_node, hospital_node, weight='travel_time'), 'travel_time'))
C:\Users\Adrian\AppData\Local\Temp\ipykernel_20956\1004238746.py:70: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `routing.route_to_gdf` function instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  travel_time = sum(ox.utils_graph.get_route_edge_attributes(G, ox.shortest_path(G, street_node, hospital_node, weight='travel_time'), 'travel_time'))
C:\Users\Adrian\AppData\Local\Temp\ipykernel_20956\1004238746.py:7

Error processing Roman,Neamt: No data elements in server response. Check log and query location/tags.
Error processing Voluntari,Bucuresti: Found no graph nodes within the requested polygon


C:\Users\Adrian\AppData\Local\Temp\ipykernel_20956\1004238746.py:70: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `routing.route_to_gdf` function instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  travel_time = sum(ox.utils_graph.get_route_edge_attributes(G, ox.shortest_path(G, street_node, hospital_node, weight='travel_time'), 'travel_time'))
C:\Users\Adrian\AppData\Local\Temp\ipykernel_20956\1004238746.py:70: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `routing.route_to_gdf` function instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  travel_time = sum(ox.utils_graph.get_route_edge_attributes(G, ox.shortest_path(G, street_node, hospital_node, weight='travel_time'), 'travel_time'))
C:\Users\Adrian\AppData\Local\Temp\ipykernel_20956\1004238746.py:7

Error processing Medgidia,Constanta: No data elements in server response. Check log and query location/tags.


C:\Users\Adrian\AppData\Local\Temp\ipykernel_20956\1004238746.py:70: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `routing.route_to_gdf` function instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  travel_time = sum(ox.utils_graph.get_route_edge_attributes(G, ox.shortest_path(G, street_node, hospital_node, weight='travel_time'), 'travel_time'))
C:\Users\Adrian\AppData\Local\Temp\ipykernel_20956\1004238746.py:70: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `routing.route_to_gdf` function instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  travel_time = sum(ox.utils_graph.get_route_edge_attributes(G, ox.shortest_path(G, street_node, hospital_node, weight='travel_time'), 'travel_time'))
C:\Users\Adrian\AppData\Local\Temp\ipykernel_20956\1004238746.py:7

## GET BEDS/1000 people ratio for each city

In [ ]:
import pandas as pd
import networkx as nx
import geopandas as gpd
import osmnx as ox

# Create empty lists to store data
city_names = []
populations = []
bed_counts_list = []
beds_population_ratios = []

for i in range(1, len(city_name)):
    try:
        location = city_name[i] + "," + county_name[i]
        population = city_population[i]
        # Get the hospitals data
        hospitals = ox.features.features_from_place(location, tags={"amenity": "hospital", "amenity":"clinic", "healthcare":"hospital", "emergency":"yes"})

        # Load the XLS file using pandas
        county_xls = "judetul-" + county_name[i].lower() + '.xls'
        xls_file_path = 'Datasets/HospitalsPerCounty/' + county_xls  # Replace with your actual file path

        with open(xls_file_path, 'rb') as file:
            xls = pd.ExcelFile(file)

            # Read the specific sheet
            df = pd.read_excel(file, sheet_name=xls.sheet_names[0], header=None)

        # Extract hospital names and bed counts
        hospital_names = []
        bed_counts = []

        for index, row in df.iterrows():
            if pd.notnull(row[1]) and row[1].startswith("U.SPIT."):
                hospital_name = row[1]
                if pd.notnull(row[2]):  # Check if beds information is in the same row
                    bed_counts.append(row[2])
                else:  # Beds information might be in the next row
                    bed_counts.append(df.iloc[index + 1, 2])
                    hospital_name += ' ' + str(df.iloc[index + 1, 1])  # Save the next row as part of the hospital name
                # Remove unwanted spaces in the hospital name
                hospital_name = ' '.join(hospital_name.split())

                # Replace special characters
                hospital_name = hospital_name.replace('Ț', 'T')
                hospital_name = hospital_name.replace('Ţ', 'T')
                hospital_name = hospital_name.replace('Ă', 'A')
                hospital_name = hospital_name.replace('Ș', 'S')
                hospital_name = hospital_name.replace('Ş', 'S')
                hospital_name = hospital_name.replace('Î', 'I')
                hospital_name = hospital_name.replace('Â', 'A')

                hospital_names.append(hospital_name)

        # Combine hospital names and bed counts
        combined_df = pd.DataFrame({'Hospital': hospital_names, 'Beds': bed_counts})

        number_of_beds = 0
        # Calculate total number of beds
        for index, row in combined_df.iterrows():
            if city_name[i].upper() in row['Hospital']:
                number_of_beds = number_of_beds + row['Beds']

        # Calculate beds per 1000 people
        beds_population_ratio = float(number_of_beds) * 1000 / float(population)

        # Store data in lists
        city_names.append(city_name[i])
        populations.append(population)
        bed_counts_list.append(number_of_beds)
        beds_population_ratios.append(beds_population_ratio)

    except Exception as e:
        print(f"Error processing {city_name[i]}: {e}")




Error processing Deva: [Errno 2] No such file or directory: 'Datasets/HospitalsPerCounty/judetul-hunedoara.xls'
Error processing Hunedoara: [Errno 2] No such file or directory: 'Datasets/HospitalsPerCounty/judetul-hunedoara.xls'
Error processing Roman: No data elements in server response. Check log and query location/tags.
Error processing Voluntari: No data elements in server response. Check log and query location/tags.
Error processing Medgidia: No data elements in server response. Check log and query location/tags.
Error processing Petrosani: [Errno 2] No such file or directory: 'Datasets/HospitalsPerCounty/judetul-hunedoara.xls'


## WRITE DATA IN CSV FILE

In [ ]:
def max_size(a, b, c, d, e, f):
    return max(len(a), max(len(b), max(len(c), max(len(d), max(len(e), len(f))))))

max_s = max_size(city_names, populations, average_shortest_path, average_shortest_time, bed_counts_list, beds_population_ratios)



In [ ]:
# Create DataFrame from lists
print(len(city_names), len(populations), len(average_shortest_path), len(average_shortest_time), len(bed_counts_list), len(beds_population_ratios))
data = {
    'city_name': city_names,
    'population': populations,
    'average_distance': average_shortest_path,
    'average_time': average_shortest_time,
    'number_of_beds': bed_counts_list,
    'bed_population_ratio': beds_population_ratios
}
df = pd.DataFrame(data)

# Write DataFrame to CSV file
df.to_csv('Datasets\HospitalData\hospital_data.csv', index=False)

print("Data has been written to hospital_data.csv file.")

69 69 69 69 69 69
Data has been written to hospital_data.csv file.


<>:14: DeprecationWarning: invalid escape sequence '\H'
<>:14: DeprecationWarning: invalid escape sequence '\H'
C:\Users\Adrian\AppData\Local\Temp\ipykernel_20956\604347627.py:14: DeprecationWarning: invalid escape sequence '\H'
  df.to_csv('Datasets\HospitalData\hospital_data.csv', index=False)
